In [1]:
#Cifar image classification using TF 2.0 Keras

In [2]:
import tensorflow as tf
from tensorflow import keras
import datetime as dt

tf.__version__

E:\Continuum\anaconda3\envs\tf2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'2.0.0-alpha0'

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(10000)
train_dataset = train_dataset.map(lambda x, y: (tf.math.divide(tf.cast(x, tf.float32), 255.0), tf.reshape(tf.one_hot(y, 10), (-1, 10))))
train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
train_dataset = train_dataset.repeat()

In [4]:
valid_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1000).shuffle(10000)

In [5]:
valid_dataset = valid_dataset.map(lambda x, y: (tf.math.divide(tf.cast(x, tf.float32),255.0), tf.reshape(tf.one_hot(y, 10), (-1, 10))))


In [6]:

valid_dataset = valid_dataset.repeat()

In [7]:
class CIFAR10Model(keras.Model):
    def __init__(self):
        super(CIFAR10Model, self).__init__(name='cifar_cnn')
        self.conv1 = keras.layers.Conv2D(64, 5,activation=tf.nn.relu)                             
                                         
        self.max_pool2d = keras.layers.MaxPooling2D((3, 3), (2, 2))
        self.max_norm = keras.layers.BatchNormalization()
        self.conv2 = keras.layers.Conv2D(64, 5,activation=tf.nn.relu)                 
                                        
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(750, activation=tf.nn.relu)
                                      
        self.fc2 = keras.layers.Dense(10)
        self.softmax = keras.layers.Softmax()

    def call(self, x):
        x = self.max_pool2d(self.conv1(x))
        x = self.max_norm(x)
        x = self.max_pool2d(self.conv2(x))
        x = self.max_norm(x)
        x = self.flatten(x)
        x = self.fc2(x)
        return self.softmax(x)

In [8]:
model = CIFAR10Model()

In [9]:

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(train_dataset, epochs=2, steps_per_epoch=100,
          validation_data=valid_dataset,
          validation_steps=3)


Epoch 1/2
100/100 [==============================] - 17s 170ms/step - loss: 1.9738 - accuracy: 0.2642 - val_loss: 3.4502 - val_accuracy: 0.1250
Epoch 2/2
100/100 [==============================] - 14s 140ms/step - loss: 1.6112 - accuracy: 0.4335 - val_loss: 3.9630 - val_accuracy: 0.1193
